In [6]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

In [7]:
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=2, memory_limit='1GB')
client

/Users/samypalaniappan/opt/miniconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:50538 Dashboard: http://127.0.0.1:50539/status,Cluster Workers: 4 Cores: 8 Memory: 4.00 GB


In [22]:
df = dd.read_csv("electronics.csv", 
                 blocksize=64000000,
                 storage_options=dict(anon=True),
                 assume_missing=True
                )
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,41409413.0,R2MTG1GCZLR2DK,B00428R89M,112201306.0,yoomall 5M Antenna WIFI RP-SMA Female to Male ...,Electronics,5.0,0.0,0.0,N,Y,Five Stars,As described.,2015-08-31
1,US,49668221.0,R2HBOEM8LE9928,B000068O48,734576678.0,Hosa GPM-103 3.5mm TRS to 1/4inch TRS Adaptor,Electronics,5.0,0.0,0.0,N,Y,It works as advertising.,It works as advertising.,2015-08-31
2,US,12338275.0,R1P4RW1R9FDPEE,B000GGKOG8,614448099.0,Channel Master Titan 2 Antenna Preamplifier,Electronics,5.0,1.0,1.0,N,Y,Five Stars,Works pissa,2015-08-31
3,US,38487968.0,R1EBPM82ENI67M,B000NU4OTA,72265257.0,LIMTECH Wall charger + USB Hotsync & Charging ...,Electronics,1.0,0.0,0.0,N,Y,One Star,Did not work at all.,2015-08-31
4,US,23732619.0,R372S58V6D11AT,B00JOQIO6S,308169188.0,Skullcandy Air Raid Portable Bluetooth Speaker,Electronics,5.0,1.0,1.0,N,Y,Overall pleased with the item,Works well. Bass is somewhat lacking but is pr...,2015-08-31


In [23]:
df.npartitions

27

In [24]:
type(df)

dask.dataframe.core.DataFrame

In [27]:
df.star_rating.value_counts(split_out=1).compute()

5.0    1781161
4.0     536821
1.0     358120
3.0     238587
2.0     179180
Name: star_rating, dtype: int64

In [28]:
filereader = pd.read_csv('electronics.csv', iterator=True, chunksize=1000)

In [31]:
for _ in range(3093):
    chunk = filereader.get_chunk()
    chunk['sentiment'] = np.where(chunk['star_rating'] >= 4, 'positive', 'negative')
chunk.head()

StopIteration: 